In [69]:
%%time

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 21 µs


In [155]:
!pip install --upgrade https://github.com/Lasagne/Lasagne/archive/master.zip

     | 235kB 769kB/s
Requirement already up-to-date: numpy in /Applications/anaconda/lib/python3.5/site-packages (from Lasagne==0.2.dev1)
  Found existing installation: Lasagne 0.1
    Uninstalling Lasagne-0.1:
      Successfully uninstalled Lasagne-0.1
  Running setup.py install for Lasagne ... - \ | done


# theano teaser

Doing the very same thing

In [86]:
import data_helpers

tweets='TREC_all.txt'
labels='label_all.txt'

X,y=data_helpers.load_tweets(tweets),data_helpers.load_2dsentiment(labels)

In [83]:
y_check=data_helpers.load_1dsentiment(labels)

In [84]:
a=0
b=0
for i in y_check:
    if i:
        a+=1
    else:
        b+=1
print(a,b)

2421 6971


In [87]:
import numpy as np
import theano
import theano.tensor as T
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.contrib import learn
import lasagne
from sklearn.model_selection import train_test_split


from sklearn.metrics import roc_auc_score

In [88]:
max_document_length = max([len(x.split(" ")) for x in X])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x_bag = np.array(list(vocab_processor.fit_transform(X)))

In [116]:
len(vocab_processor.vocabulary_)

13028

In [89]:
#Check if balance
X_train, X_test, y_train, y_test = train_test_split(x_bag, y, test_size=0.33, random_state=42)

print ("Train class frequencies:\t", [col.nonzero()[0].shape[0] for col in y_train.transpose()])
print ("Validation class frequencies:\t", [col.nonzero()[0].shape[0] for col in y_test.transpose()])
print ("Constant classifier's validation accuracy:\t", [col.nonzero()[0].shape[0] for col in y_test.transpose()][1] * 1. / y_test.shape[0])

Train class frequencies:	 [1639, 4653]
Validation class frequencies:	 [782, 2318]
Constant classifier's validation accuracy:	 0.747741935483871


In [90]:
X_train, X_val = X_train[:-5500], X_train[-5500:]
y_train, y_val = y_train[:-5500], y_train[-5500:]

### That's all, now let's train it!
* We got a lot of data, so it's recommended that you use SGD
* So let's implement a function that splits the training sample into minibatches

In [91]:
# An auxilary function that returns mini-batches for neural network training

#Parameters
# X - a tensor of images with shape (many, 1, 28, 28), e.g. X_train
# y - a vector of answers for corresponding images e.g. Y_train
#batch_size - a single number - the intended size of each batches


def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

# Training loop

# A better network 
 

## Tips on what can be done:



 * Network size
   * MOAR neurons, 
   * MOAR layers, 
   * Convolutions are almost imperative
   * Пх'нглуи мглв'нафх Ктулху Р'льех вгах'нагл фхтагн! 
   
   
   
 * Regularize to prevent overfitting
   * Add some L2 weight norm to the loss function, theano will do the rest
   * Can be done manually or via - http://lasagne.readthedocs.org/en/latest/modules/regularization.html
   
   
   
 * Better optimization - rmsprop, nesterov_momentum, adadelta, adagrad and so on.
   * Converge faster and sometimes reach better optima
   * It might make sense to tweak learning rate, other learning parameters, batch size and number of epochs
   
   
   
 * Dropout - to prevent overfitting
   * `lasagne.layers.DropoutLayer(prev_layer, p=probability_to_zero_out)`
   
   
   
 * Convolution layers
   * `network = lasagne.layers.Conv2DLayer(prev_layer,`
    `                       num_filters = n_neurons,`
    `                        filter_size = (filter width, filter height),`
    `                        nonlinearity = some_nonlinearity)`
   * Warning! Training convolutional networks can take long without GPU.
     * If you are CPU-only, we still recomment to try a simple convolutional architecture
     * a perfect option is if you can set it up to run at nighttime and check it up at the morning.
 
 * Plenty other layers and architectures
   * http://lasagne.readthedocs.org/en/latest/modules/layers.html
   * batch normalization, pooling, etc
   
   
 * Nonlinearities in the hidden layers
   * tanh, relu, leaky relu, etc
   
 
 
   
There is a template for your solution below that you can opt to use or throw away and write it your way

In [99]:
input_X = T.lmatrix("X")

#input dimention (None means "Arbitrary" and only works at  the first axes [samples])
input_shape = (None, 39)

target_y = T.vector("target Y integer",dtype='int64')



#Input layer (auxilary)
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

emb_layer=lasagne.layers.EmbeddingLayer(input_layer,len(vocab_processor.vocabulary_),50)

dim_layer=lasagne.layers.DimshuffleLayer(emb_layer, [0,2,1])

#lstm=lasagne.layers.LSTMLayer(dense_1,num_units=4)


conv_1=lasagne.layers.Conv1DLayer(dim_layer, num_filters=3, filter_size=5)#,nonlinearity=lasagne.nonlinearities.softmax)
pool_1=lasagne.layers.MaxPool1DLayer(conv_1, pool_size=2)



#drop_1=lasagne.layers.DropoutLayer(input_layer,p=0.5)
dense_1 = lasagne.layers.DenseLayer(pool_1,num_units=50)#,
                                  #nonlinearity = lasagne.nonlinearities.softmax,
#                                 name = "selu")

#conv_2=lasagne.layers.Conv1DLayer(input_layer, num_filters=3, filter_size=5)#,nonlinearity=lasagne.nonlinearities.softmax)
#pool_2=lasagne.layers.Pool2DLayer(conv_2, pool_size=2)


#drop_1=lasagne.layers.DropoutLayer(dense_1,p=0.5)

#dense_2 = lasagne.layers.DenseLayer(dense_1,num_units=100,
#                                   nonlinearity = lasagne.nonlinearities.selu,
   #                               name = "selu")

#dense_3 = lasagne.layers.DenseLayer(dense_2,num_units=100,
#                                   nonlinearity = lasagne.nonlinearities.selu,
#                                  name = "selu")


#fully connected output layer that takes dense_1 as input and has 10 neurons (1 for each digit)
#We use softmax nonlinearity to make probabilities add up to 1
dense_output = lasagne.layers.DenseLayer(dense_1,num_units = 2,
                                        nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output')



In [163]:
# Network predictions (theano-transformation)
y_predicted = lasagne.layers.get_output(dense_output,input_X)


#All weights (shared-varaibles)
# "trainable" flag means not to return auxilary params like batch mean (for batch normalization)
all_weights = lasagne.layers.get_all_params(dense_output,trainable=True)
print (all_weights)

[W, W, b, W, b, output.W, output.b]


In [164]:
#Mean categorical crossentropy as a loss function - similar to logistic loss but for multiclass targets
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()



from lasagne.regularization import regularize_layer_params_weighted, l2, l1
from lasagne.regularization import regularize_layer_params
layers = {conv_1: 0.1, dense_1: 0.5}
l2_penalty = regularize_layer_params_weighted(layers, l2)
l1_penalty = regularize_layer_params(dense_1, l1) * 1e-4
#loss=loss+l2_penalty#+l1_penalty

#prediction accuracy
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()

#This function computes gradient AND composes weight updates just like you did earlier
updates_sgd = lasagne.updates.adagrad(loss, all_weights,learning_rate=0.01)


#A function that accepts X and y, returns loss functions and performs weight updates
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)

#A function that just computes accuracy given X and y
accuracy_fun = theano.function([input_X,target_y],accuracy)

In [166]:
#итерации обучения
import time
num_epochs = 100#<how many times to iterate over the entire training set>

batch_size = 50#<how many samples are processed at a single function call>

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

Epoch 1 of 100 took 0.725s
  training loss (in-iteration):		0.575040
  train accuracy:		70.67 %
  validation accuracy:		75.95 %
Epoch 2 of 100 took 0.502s
  training loss (in-iteration):		0.332779
  train accuracy:		87.07 %
  validation accuracy:		83.62 %
Epoch 3 of 100 took 0.647s
  training loss (in-iteration):		0.166882
  train accuracy:		95.07 %
  validation accuracy:		84.42 %
Epoch 4 of 100 took 0.464s
  training loss (in-iteration):		0.080947
  train accuracy:		98.13 %
  validation accuracy:		84.33 %
Epoch 5 of 100 took 0.544s
  training loss (in-iteration):		0.042614
  train accuracy:		99.20 %
  validation accuracy:		84.18 %
Epoch 6 of 100 took 0.462s
  training loss (in-iteration):		0.025954
  train accuracy:		99.47 %
  validation accuracy:		84.20 %
Epoch 7 of 100 took 0.475s
  training loss (in-iteration):		0.017441
  train accuracy:		99.60 %
  validation accuracy:		84.07 %
Epoch 8 of 100 took 0.466s
  training loss (in-iteration):		0.012304
  train accuracy:		99.73 %
  valida

In [274]:
test_acc = 0
test_batches = 0
f1score=0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 99:
    print ("Achievement unlocked: 80lvl Warlock!")
else:
    print ("We need more magic!")

Final results:
  test accuracy:		86.00 %
We need more magic!


In [94]:
import lasagne

input_X = T.lmatrix("X")

#input dimention (None means "Arbitrary" and only works at  the first axes [samples])
input_shape = (None, 39)

target_y = T.matrix("target Y integer",dtype='int64')

#Input layer (auxilary)
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

emb_layer=lasagne.layers.EmbeddingLayer(input_layer,len(vocab_processor.vocabulary_),50)

dim_layer=lasagne.layers.DimshuffleLayer(emb_layer, [0,2,1])

conv_1=lasagne.layers.Conv1DLayer(dim_layer, num_filters=3, filter_size=5)#,nonlinearity=lasagne.nonlinearities.softmax)
pool_1=lasagne.layers.MaxPool1DLayer(conv_1, pool_size=2)



#drop_1=lasagne.layers.DropoutLayer(input_layer,p=0.5)
dense_1 = lasagne.layers.DenseLayer(pool_1,num_units=50)#,
                                  #nonlinearity = lasagne.nonlinearities.softmax,
#                                 name = "selu")


dense_output = lasagne.layers.DenseLayer(dense_1,num_units = 2,
                                        nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output')

y_predicted = lasagne.layers.get_output(dense_output,input_X)


#All weights (shared-varaibles)
# "trainable" flag means not to return auxilary params like batch mean (for batch normalization)
all_weights = lasagne.layers.get_all_params(dense_output,trainable=True)
print (all_weights)

loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()



from lasagne.regularization import regularize_layer_params_weighted, l2, l1
from lasagne.regularization import regularize_layer_params
layers = {conv_1: 0.1, dense_1: 0.5}
l2_penalty = regularize_layer_params_weighted(layers, l2)
l1_penalty = regularize_layer_params(dense_1, l1) * 1e-4
#loss=loss+l2_penalty#+l1_penalty

#prediction accuracy
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()

#This function computes gradient AND composes weight updates just like you did earlier
updates_sgd = lasagne.updates.adagrad(loss, all_weights,learning_rate=0.01)

train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)

#A function that just computes accuracy given X and y
accuracy_fun = theano.function([input_X,target_y],accuracy)

#итерации обучения
import time
num_epochs = 20#<how many times to iterate over the entire training set>

batch_size = 50#<how many samples are processed at a single function call>

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    f_tr_macro=0
    f_tr_micro=0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        f_tr_micro+=f1_score(targets, predict_fun(inputs), average='micro')
        f_tr_macro+=f1_score(targets, predict_fun(inputs), average='macro')
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    val_fscore_micro=0
    val_fscore_macro=0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_fscore_micro+=f1_score(targets, predict_fun(inputs), average='micro')
        val_fscore_macro+=f1_score(targets, predict_fun(inputs), average='macro')
        
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))
    print("  train f1-score-macro:\t\t{:.2f} %".format(
        f_tr_macro / train_batches * 100))
    print("  train f1-score-micro:\t\t{:.2f} %".format(
        f_tr_micro / train_batches * 100))
    print("  val f1-score-macro:\t\t{:.2f} %".format(
        val_fscore_macro / val_batches * 100))
    print("  val f1-score-micro:\t\t{:.2f} %".format(
        val_fscore_micro / val_batches * 100))

[W, W, b, W, b, output.W, output.b]


TypeError: Bad input argument to theano function with name "<ipython-input-80-f35522de8c28>:89" at index 0 (0-based).  
Backtrace when that variable is created:

  File "/Applications/anaconda/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 228, in dispatch_shell
    handler(stream, idents, msg)
  File "/Applications/anaconda/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 390, in execute_request
    user_expressions, allow_stdin)
  File "/Applications/anaconda/lib/python3.5/site-packages/ipykernel/ipkernel.py", line 196, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/Applications/anaconda/lib/python3.5/site-packages/ipykernel/zmqshell.py", line 501, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/Applications/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2717, in run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  

In [ ]:
test_acc = 0
test_batches = 0
test_fscore_micro=0
test_fscore_macro=0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
    test_fscore_micro+=f1_score(targets, predict_fun(inputs), average='micro')
    test_fscore_macro+=f1_score(targets, predict_fun(inputs), average='macro')
        
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))
print("  test f1-score-macro:\t\t{:.2f} %".format(
        test_fscore_macro / test_batches * 100))
print("  test f1-score-micro:\t\t{:.2f} %".format(
        test_fscore_micro / test_batches * 100))

if test_acc / test_batches * 100 > 99:
    print ("Achievement unlocked: 80lvl Warlock!")
else:
    print ("We need more magic!")

In [270]:
from sklearn.metrics import f1_score


In [227]:
input_X = T.lmatrix("X")

#input dimention (None means "Arbitrary" and only works at  the first axes [samples])
input_shape = (None, 39)

target_y = T.vector("target Y integer",dtype='int64')


#Input layer (auxilary)
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

emb_layer=lasagne.layers.EmbeddingLayer(input_layer,len(vocab_processor.vocabulary_),50)

dim_layer=lasagne.layers.DimshuffleLayer(emb_layer, [0,2,1])

#lstm=lasagne.layers.LSTMLayer(dense_1,num_units=4)


conv_1=lasagne.layers.Conv1DLayer(dim_layer, num_filters=3, filter_size=5)#,nonlinearity=lasagne.nonlinearities.softmax)
pool_1=lasagne.layers.MaxPool1DLayer(conv_1, pool_size=2)



#drop_1=lasagne.layers.DropoutLayer(pool_1,p=0.5)
dense_1 = lasagne.layers.DenseLayer(pool_1,num_units=50)#,
                                  #nonlinearity = lasagne.nonlinearities.softmax,
#                                 name = "selu")

dense_output = lasagne.layers.DenseLayer(dense_1,num_units = 2,
                                        nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output')


# Network predictions (theano-transformation)
y_predicted = lasagne.layers.get_output(dense_output,input_X)


#All weights (shared-varaibles)
# "trainable" flag means not to return auxilary params like batch mean (for batch normalization)
all_weights = lasagne.layers.get_all_params(dense_output,trainable=True)
print (all_weights)


#Mean categorical crossentropy as a loss function - similar to logistic loss but for multiclass targets
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()



from lasagne.regularization import regularize_layer_params_weighted, l2, l1
from lasagne.regularization import regularize_layer_params
layers = {conv_1: 0.1, dense_1: 0.5}
l2_penalty = regularize_layer_params_weighted(layers, l2)
l1_penalty = regularize_layer_params(dense_1, l1) * 1e-4
loss=loss+l1_penalty#+l1_penalty

#prediction accuracy
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()

#This function computes gradient AND composes weight updates just like you did earlier
updates_sgd = lasagne.updates.adamax(loss, all_weights,learning_rate=0.01)



#A function that accepts X and y, returns loss functions and performs weight updates
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)

#A function that just computes accuracy given X and y
accuracy_fun = theano.function([input_X,target_y],accuracy)



#итерации обучения
import time
num_epochs = 10#<how many times to iterate over the entire training set>

batch_size = 50#<how many samples are processed at a single function call>

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    f_macro=0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

[W, W, b, W, b, output.W, output.b]
Epoch 1 of 10 took 0.337s
  training loss (in-iteration):		0.533203
  train accuracy:		71.87 %
  validation accuracy:		83.13 %
Epoch 2 of 10 took 0.378s
  training loss (in-iteration):		0.250634
  train accuracy:		90.80 %
  validation accuracy:		85.35 %
Epoch 3 of 10 took 0.418s
  training loss (in-iteration):		0.103816
  train accuracy:		97.60 %
  validation accuracy:		85.45 %
Epoch 4 of 10 took 0.354s
  training loss (in-iteration):		0.043320
  train accuracy:		99.20 %
  validation accuracy:		85.53 %
Epoch 5 of 10 took 0.351s
  training loss (in-iteration):		0.022692
  train accuracy:		99.87 %
  validation accuracy:		85.38 %
Epoch 6 of 10 took 0.345s
  training loss (in-iteration):		0.016745
  train accuracy:		100.00 %
  validation accuracy:		85.18 %
Epoch 7 of 10 took 0.351s
  training loss (in-iteration):		0.015013
  train accuracy:		100.00 %
  validation accuracy:		85.25 %
Epoch 8 of 10 took 0.347s
  training loss (in-iteration):		0.014116
  tra

In [228]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
    f_macro=f1_score(batch[1],)
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 99:
    print ("Achievement unlocked: 80lvl Warlock!")
else:
    print ("We need more magic!")

Final results:
  test accuracy:		86.00 %
We need more magic!


In [242]:
pool_1.output_shape

(None, 3, 17)

In [265]:
input_X = T.lmatrix("X")

#input dimention (None means "Arbitrary" and only works at  the first axes [samples])
input_shape = (None, 39)

target_y = T.vector("target Y integer",dtype='int64')


#Input layer (auxilary)
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

emb_layer=lasagne.layers.EmbeddingLayer(input_layer,len(vocab_processor.vocabulary_),50)

dim_layer=lasagne.layers.DimshuffleLayer(emb_layer, [0,2,1])

#lstm=lasagne.layers.LSTMLayer(dense_1,num_units=4)


conv_1=lasagne.layers.Conv1DLayer(dim_layer, num_filters=3, filter_size=5)#,nonlinearity=lasagne.nonlinearities.softmax)
pool_1=lasagne.layers.MaxPool1DLayer(conv_1, pool_size=2)

"""dim_layer_2=lasagne.layers.DimshuffleLayer(pool_1, [0,2,1])
conv_2=lasagne.layers.Conv1DLayer(dim_layer_2, num_filters=3, filter_size=2)#,nonlinearity=lasagne.nonlinearities.softmax)
pool_2=lasagne.layers.MaxPool1DLayer(conv_2, pool_size=2)"""



#drop_1=lasagne.layers.DropoutLayer(pool_1,p=0.5)
dense_1 = lasagne.layers.DenseLayer(pool_1,num_units=50)#,
                                  #nonlinearity = lasagne.nonlinearities.softmax,
#                                 name = "selu")


drop_1=lasagne.layers.DropoutLayer(dense_1,p=0.5)

dense_2 = lasagne.layers.DenseLayer(drop_1,num_units=100,
 #                                  nonlinearity = lasagne.nonlinearities.softmax,
                                  name = "selu")

#dense_3 = lasagne.layers.DenseLayer(dense_2,num_units=100,
#                                   nonlinearity = lasagne.nonlinearities.selu,
#                                  name = "selu")


#fully connected output layer that takes dense_1 as input and has 10 neurons (1 for each digit)
#We use softmax nonlinearity to make probabilities add up to 1
dense_output = lasagne.layers.DenseLayer(dense_2,num_units = 2,
                                        nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output')


# Network predictions (theano-transformation)
y_predicted = lasagne.layers.get_output(dense_output,input_X)


#All weights (shared-varaibles)
# "trainable" flag means not to return auxilary params like batch mean (for batch normalization)
all_weights = lasagne.layers.get_all_params(dense_output,trainable=True)
print (all_weights)


#Mean categorical crossentropy as a loss function - similar to logistic loss but for multiclass targets
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()



from lasagne.regularization import regularize_layer_params_weighted, l2, l1
from lasagne.regularization import regularize_layer_params
layers = {conv_1: 0.1, dense_1: 0.5}
l2_penalty = regularize_layer_params_weighted(layers, l2)
l1_penalty = regularize_layer_params(dense_1, l1) * 1e-4
#loss=loss+l1_penalty#+l1_penalty

#prediction accuracy
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()

#This function computes gradient AND composes weight updates just like you did earlier
updates_sgd = lasagne.updates.adamax(loss, all_weights,learning_rate=0.01)

y_binary=theano.tensor.gt(y_predicted, 0.5)
predict_fun=theano.function([input_X],y_binary)

#A function that accepts X and y, returns loss functions and performs weight updates
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)

#A function that just computes accuracy given X and y
accuracy_fun = theano.function([input_X,target_y],accuracy)



#итерации обучения
import time
num_epochs = 100#<how many times to iterate over the entire training set>

batch_size = 50#<how many samples are processed at a single function call>

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    f_macro=0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

[W, W, b, W, b, selu.W, selu.b, output.W, output.b]
Epoch 1 of 100 took 0.400s
  training loss (in-iteration):		0.599808
  train accuracy:		72.80 %
  validation accuracy:		74.05 %
Epoch 2 of 100 took 0.397s
  training loss (in-iteration):		0.579688
  train accuracy:		72.93 %
  validation accuracy:		74.05 %
Epoch 3 of 100 took 0.399s
  training loss (in-iteration):		0.532689
  train accuracy:		72.93 %
  validation accuracy:		74.05 %
Epoch 4 of 100 took 0.392s
  training loss (in-iteration):		0.394204
  train accuracy:		75.73 %
  validation accuracy:		78.49 %
Epoch 5 of 100 took 0.396s
  training loss (in-iteration):		0.282482
  train accuracy:		90.93 %
  validation accuracy:		81.93 %
Epoch 6 of 100 took 0.393s
  training loss (in-iteration):		0.191221
  train accuracy:		95.47 %
  validation accuracy:		81.62 %
Epoch 7 of 100 took 0.396s
  training loss (in-iteration):		0.099631
  train accuracy:		97.60 %
  validation accuracy:		80.75 %
Epoch 8 of 100 took 0.394s
  training loss (in-itera

KeyboardInterrupt: 

In [257]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 99:
    print ("Achievement unlocked: 80lvl Warlock!")
else:
    print ("We need more magic!")

Final results:
  test accuracy:		82.47 %
We need more magic!


Report

All creative approaches are highly welcome, but at the very least it would be great to mention
* the idea;
* brief history of tweaks and improvements;
* what is the final architecture and why?
* what is the training method and, again, why?
* Any regularizations and other techniques applied and their effects;


There is no need to write strict mathematical proofs (unless you want to).
 * "I tried this, this and this, and the second one turned out to be better. And i just didn't like the name of that one" - OK, but can be better
 * "I have analized these and these articles|sources|blog posts, tried that and that to adapt them to my problem and the conclusions are such and such" - the ideal one
 * "I took that code that demo without understanding it, but i'll never confess that and instead i'll make up some pseudoscientific explaination" - __not_ok__

In [55]:
#f1-score
from sklearn.metrics import f1_score


In [183]:
input_X = T.tensor3("X")

#input dimention (None means "Arbitrary" and only works at  the first axes [samples])
input_shape = (None, 39)

target_y = T.vector("target Y integer",dtype='int64')

N_HIDDEN = 100
# Number of training sequences in each batch
N_BATCH = 50
# Optimization learning rate
LEARNING_RATE = .001
# All gradients above this will be clipped
GRAD_CLIP = 100

In [189]:
l_in = lasagne.layers.InputLayer(shape=(N_BATCH, 39, 1), input_var=input_X)#len(vocab_processor.vocabulary_)
    # The network also needs a way to provide a mask for each sequence.  We'll
    # use a separate input layer for that.  Since the mask only determines
    # which indices are part of the sequence for each batch entry, they are
    # supplied as matrices of dimensionality (N_BATCH, MAX_LENGTH)
l_forward = lasagne.layers.RecurrentLayer(
        l_in, N_HIDDEN, grad_clipping=GRAD_CLIP,
        W_in_to_hid=lasagne.init.HeUniform(),
        W_hid_to_hid=lasagne.init.HeUniform(),
        nonlinearity=lasagne.nonlinearities.tanh, only_return_final=True)
l_backward = lasagne.layers.RecurrentLayer(
        l_in, N_HIDDEN, grad_clipping=GRAD_CLIP,
        W_in_to_hid=lasagne.init.HeUniform(),
        W_hid_to_hid=lasagne.init.HeUniform(),
        nonlinearity=lasagne.nonlinearities.tanh,
        only_return_final=True, backwards=True)
    # Now, we'll concatenate the outputs to combine them.
l_concat = lasagne.layers.ConcatLayer([l_forward, l_backward])
    # Our output layer is a simple dense connection, with 1 output unit
l_out = lasagne.layers.DenseLayer(
        l_concat, num_units=2, nonlinearity=lasagne.nonlinearities.tanh)


In [190]:
# Network predictions (theano-transformation)
y_predicted = lasagne.layers.get_output(l_out,input_X)


#All weights (shared-varaibles)
# "trainable" flag means not to return auxilary params like batch mean (for batch normalization)
all_weights = lasagne.layers.get_all_params(l_out,trainable=True)
print (all_weights)

[input_to_hidden.W, input_to_hidden.b, hidden_to_hidden.W, input_to_hidden.W, input_to_hidden.b, hidden_to_hidden.W, W, b]


In [191]:
#Mean categorical crossentropy as a loss function - similar to logistic loss but for multiclass targets
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()



from lasagne.regularization import regularize_layer_params_weighted, l2, l1
from lasagne.regularization import regularize_layer_params
layers = {conv_1: 0.1, dense_1: 0.5}
l2_penalty = regularize_layer_params_weighted(layers, l2)
l1_penalty = regularize_layer_params(dense_1, l1) * 1e-4
#loss=loss+l2_penalty#+l1_penalty

#prediction accuracy
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()

#This function computes gradient AND composes weight updates just like you did earlier
updates_sgd = lasagne.updates.adagrad(loss, all_weights,learning_rate=0.01)

In [192]:
#A function that accepts X and y, returns loss functions and performs weight updates
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)

#A function that just computes accuracy given X and y
accuracy_fun = theano.function([input_X,target_y],accuracy)

In [199]:
X_test_r = X_test.reshape(len(X_test),39, 1)
X_train_r=X_train.reshape(len(X_train),39, 1)
X_val_r= X_val.reshape(len(X_val),39, 1)

In [200]:
#итерации обучения
import time
num_epochs = 100#<how many times to iterate over the entire training set>

batch_size = 50#<how many samples are processed at a single function call>

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train_r, y_train,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val_r, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

Epoch 1 of 100 took 1.881s
  training loss (in-iteration):		nan
  train accuracy:		68.27 %
  validation accuracy:		66.65 %
Epoch 2 of 100 took 1.789s
  training loss (in-iteration):		nan
  train accuracy:		65.87 %
  validation accuracy:		66.65 %
Epoch 3 of 100 took 2.312s
  training loss (in-iteration):		nan
  train accuracy:		65.87 %
  validation accuracy:		66.65 %
Epoch 4 of 100 took 2.103s
  training loss (in-iteration):		nan
  train accuracy:		65.87 %
  validation accuracy:		66.65 %
Epoch 5 of 100 took 1.885s
  training loss (in-iteration):		nan
  train accuracy:		65.87 %
  validation accuracy:		66.65 %
Epoch 6 of 100 took 2.399s
  training loss (in-iteration):		nan
  train accuracy:		65.87 %
  validation accuracy:		66.65 %
Epoch 7 of 100 took 2.045s
  training loss (in-iteration):		nan
  train accuracy:		65.87 %
  validation accuracy:		66.65 %
Epoch 8 of 100 took 1.866s
  training loss (in-iteration):		nan
  train accuracy:		65.87 %
  validation accuracy:		66.65 %
Epoch 9 of 100 t

In [201]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test_r, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 99:
    print ("Achievement unlocked: 80lvl Warlock!")
else:
    print ("We need more magic!")

Final results:
  test accuracy:		68.10 %
We need more magic!


In [5]:
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import re
import pymorphy2

word_vectors = Word2Vec.load_word2vec_format('ruscorpora.model.bin', binary=True) 
morph = pymorphy2.MorphAnalyzer()
rus = stopwords.words('russian')
def get_word_variants(word, word_vectors):
               
        try:
            return word_vectors[word]   #if 'word' in word_vectors.vocab
        except KeyError:
             try:
                p = morph.parse(word)[0]
                return word_vectors[p.normal_form]
             except KeyError:
                return []

def clean_str_vec(string):
        """
    Tokenization/string cleaning for all datasets except for SST.
    """
        string = re.sub(r"[^а-яА-Я0-9(),:!?\'\`]", " ", string)
        string = re.sub(r",", " , ", string)
        string = re.sub(r"!", " ! ", string)
        string = re.sub(r"\(", " \( ", string)
        string = re.sub(r"\)", " \) ", string)
        string = re.sub(r"\?", " \? ", string)
        string = re.sub(r"\s{2,}", " ", string)
        return string.strip().lower()

def makeVec(text):  
        res=[]
        text=clean_str_vec(text).split()
        for word in text:
            if word in rus:
                pass
            else:
                res+=list(get_word_variants(word, word_vectors))
        return (res)

In [6]:
Xvec=[]
for i in X:
    Xvec.append(makeVec(i))

In [7]:
lenghts=[]
for i in Xvec:
    lenghts.append(len(i))
maximum=max(lenghts)

In [8]:
for i in range(len(Xvec)):
        while(len(Xvec[i])<maximum):
            Xvec[i].append(0)

In [9]:
Xvec=np.array(Xvec)

In [337]:
Xvec.shape

(9392, 5700)

In [10]:
Xvecre=Xvec.reshape(9392,5700,1)

In [72]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(Xvecre, y, test_size=0.33, random_state=42)
X_train2, X_val2 = X_train2[:-5500], X_train2[-5500:]
y_train2, y_val2 = y_train2[:-5500], y_train2[-5500:]

In [80]:
input_X = T.tensor3("X")

#input dimention (None means "Arbitrary" and only works at  the first axes [samples])
input_shape = (None,5700,1) #len(vocab_processor.vocabulary_)

target_y = T.matrix("target Y integer",dtype='int64')


#Input layer (auxilary)
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

#emb_layer=lasagne.layers.EmbeddingLayer(input_layer,len(vocab_processor.vocabulary_),50)

dim_layer=lasagne.layers.DimshuffleLayer(input_layer, [0,2,1])

#lstm=lasagne.layers.LSTMLayer(input_layer,num_units=4)


conv_1=lasagne.layers.Conv1DLayer(dim_layer, num_filters=3, filter_size=5)#,nonlinearity=lasagne.nonlinearities.softmax)
pool_1=lasagne.layers.MaxPool1DLayer(conv_1, pool_size=2)



#drop_1=lasagne.layers.DropoutLayer(pool_1,p=0.5)
dense_1 = lasagne.layers.DenseLayer(pool_1,num_units=300)#,
                                  #nonlinearity = lasagne.nonlinearities.softmax,
#                                 name = "selu")

dense_output = lasagne.layers.DenseLayer(dense_1,num_units = 2,
                                        nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output')


# Network predictions (theano-transformation)
y_predicted = lasagne.layers.get_output(dense_output,input_X)


#All weights (shared-varaibles)
# "trainable" flag means not to return auxilary params like batch mean (for batch normalization)
all_weights = lasagne.layers.get_all_params(dense_output,trainable=True)
print (all_weights)


#Mean categorical crossentropy as a loss function - similar to logistic loss but for multiclass targets
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()



from lasagne.regularization import regularize_layer_params_weighted, l2, l1
from lasagne.regularization import regularize_layer_params
layers = {conv_1: 0.1, dense_1: 0.2}
l2_penalty = regularize_layer_params_weighted(layers, l2)
l1_penalty = regularize_layer_params(dense_1, l1) * 1e-4
loss=loss+l1_penalty#+l1_penalty

#prediction accuracy
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()

#This function computes gradient AND composes weight updates just like you did earlier
updates_sgd = lasagne.updates.adamax(loss, all_weights,learning_rate=0.01)



#A function that accepts X and y, returns loss functions and performs weight updates
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)

#A function that just computes accuracy given X and y
accuracy_fun = theano.function([input_X,target_y],accuracy)


y_binary=theano.tensor.gt(y_predicted, 0.5)
predict_fun=theano.function([input_X],y_binary)


#итерации обучения
import time
num_epochs = 100#<how many times to iterate over the entire training set>

batch_size = 50#<how many samples are processed at a single function call>

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    f_tr_macro=0
    f_tr_micro=0
    start_time = time.time()
    for batch in iterate_minibatches(X_train2, y_train2,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        #prediction_new=
        #print(predict_fun(inputs))
        f_tr_micro+=f1_score(targets, predict_fun(inputs), average='micro')
        f_tr_macro+=f1_score(targets, predict_fun(inputs), average='macro')
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    val_fscore_micro=0
    val_fscore_macro=0
    
    for batch in iterate_minibatches(X_val2, y_val2, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_fscore_micro+=f1_score(targets, predict_fun(inputs), average='micro')
        val_fscore_macro+=f1_score(targets, predict_fun(inputs), average='macro')
        val_batches += 1
        

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))
    print("  train f1-score-macro:\t\t{:.2f} %".format(
        f_tr_macro / train_batches * 100))
    print("  train f1-score-micro:\t\t{:.2f} %".format(
        f_tr_micro / train_batches * 100))
    print("  val f1-score-macro:\t\t{:.2f} %".format(
        val_fscore_macro / val_batches * 100))
    print("  val f1-score-micro:\t\t{:.2f} %".format(
        val_fscore_micro / val_batches * 100))

[W, b, W, b, output.W, output.b]


/Applications/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 1 of 100 took 9.441s
  training loss (in-iteration):		2.045409
  train accuracy:		73.60 %
  validation accuracy:		78.02 %
  train f1-score-macro:		64.28 %
  train f1-score-micro:		77.33 %
  val f1-score-macro:		64.44 %
  val f1-score-micro:		78.02 %
Epoch 2 of 100 took 9.147s
  training loss (in-iteration):		1.065911
  train accuracy:		78.53 %
  validation accuracy:		78.18 %
  train f1-score-macro:		70.45 %
  train f1-score-micro:		79.87 %
  val f1-score-macro:		62.93 %
  val f1-score-micro:		78.18 %
Epoch 3 of 100 took 10.069s
  training loss (in-iteration):		0.808386
  train accuracy:		80.67 %
  validation accuracy:		78.93 %
  train f1-score-macro:		76.65 %
  train f1-score-micro:		83.73 %
  val f1-score-macro:		69.22 %
  val f1-score-micro:		78.93 %
Epoch 4 of 100 took 9.774s
  training loss (in-iteration):		0.722363
  train accuracy:		84.13 %
  validation accuracy:		79.38 %
  train f1-score-macro:		80.09 %
  train f1-score-micro:		86.00 %
  val f1-score-macro:		68.51 %
  val 

KeyboardInterrupt: 

In [82]:
test_acc = 0
test_batches = 0
test_fscore_micro=0
test_fscore_macro=0
for batch in iterate_minibatches(X_test2, y_test2, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
    test_fscore_micro+=f1_score(targets, predict_fun(inputs), average='micro')
    test_fscore_macro+=f1_score(targets, predict_fun(inputs), average='macro')
        
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))
print("  test f1-score-macro:\t\t{:.2f} %".format(
        test_fscore_macro / test_batches * 100))
print("  test f1-score-micro:\t\t{:.2f} %".format(
        test_fscore_micro / test_batches * 100))

if test_acc / test_batches * 100 > 99:
    print ("Achievement unlocked: 80lvl Warlock!")
else:
    print ("We need more magic!")

Final results:
  test accuracy:		81.47 %
  test f1-score-macro:		72.18 %
  test f1-score-micro:		81.47 %
We need more magic!
